# core

> Fill in a module description here

In [28]:
#| default_exp core

In [29]:
#| hide
from nbdev.showdoc import *

In [30]:
#| export
import logging

import coiled
import dask.distributed
import ee

In [31]:
ee.Authenticate()

True

In [32]:
# Initialize Earth Engine with the current GCP project ID
projects = !gcloud config get-value project
project_id = projects[0]
ee.Initialize(project=project_id)

In [33]:
#| export
class InitEarthEngine(dask.distributed.WorkerPlugin):
    def __init__(self, **kwargs):
        logging.warning('InitEarthEngine init')  # This appears in the notebook output where the cluster is initiated.
        self.kwargs = kwargs

    def setup(self, worker):
        logging.warning('InitEarthEngine setup')  # This appears in the dask cluster logs.
        logging.info('test info')
        logging.debug('test debug')
        logging.error('test error')
        import ee
        ee.Initialize(**self.kwargs)


class ClientGEE(dask.distributed.client.Client):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)


class ClusterGEE(coiled.Cluster):
    def __init__(self, **kwargs):
        logging.debug('ClusterGEE init')
        super().__init__(**kwargs)
        self.wait_for_workers(kwargs['n_workers'])
        coiled.credentials.google.send_application_default_credentials(self)

    def get_client(self):
        logging.debug('ClusterGEE get_client')
        client = super().get_client()
        client.register_plugin(InitEarthEngine())
        return client

In [34]:
cluster = ClusterGEE(
    name='test-class-cluster',
    n_workers=2,
    worker_cpu=8,
    region='us-west1',
)

Google Application Default Credentials have been written to a file on your Coiled VM(s).
These credentials will potentially be valid until explicitly revoked by running
gcloud auth application-default revoke


In [35]:
client = cluster.get_client()
client

<Client: 'tls://10.1.0.38:8786' processes=2 threads=16, memory=61.15 GiB>

In [36]:
def get_country_area(country_name):
    country = country_fc.filter(ee.Filter.eq('country_na', country_name))
    area_km2 = country.geometry().area().multiply(1e-6).round()
    return (country_name, area_km2.getInfo(), )

country_fc = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017')
country_list = country_fc.aggregate_array('country_na').distinct().sort().getInfo()

# Create a list of jobs to distribute among the workers
submitted_jobs = [
    client.submit(get_country_area, country)
    for country in country_list
]
results = client.gather(submitted_jobs)
results

[('Abyei Area', 10460),
 ('Afghanistan', 642093),
 ('Akrotiri', 127),
 ('Aksai Chin', 30448),
 ('Albania', 28638),
 ('Algeria', 2311174),
 ('American Samoa', 201),
 ('Andorra', 464),
 ('Angola', 1251602),
 ('Anguilla', 76),
 ('Antarctica', 12247200),
 ('Antigua & Barbuda', 448),
 ('Argentina', 2779304),
 ('Armenia', 29726),
 ('Aruba', 186),
 ('Ashmore & Cartier Is', 1),
 ('Australia', 7699189),
 ('Austria', 83696),
 ('Azerbaijan', 86024),
 ('Bahamas, The', 12036),
 ('Bahrain', 780),
 ('Bangladesh', 138262),
 ('Barbados', 443),
 ('Belarus', 206747),
 ('Belgium', 30544),
 ('Belize', 22129),
 ('Benin', 115867),
 ('Bermuda', 64),
 ('Bhutan', 38645),
 ('Bir Tawil', 2014),
 ('Bolivia', 1087508),
 ('Bosnia & Herzegovina', 51122),
 ('Botswana', 579654),
 ('Bouvet Island', 50),
 ('Brazil', 8503009),
 ('British Indian Ocean Terr', 52),
 ('British Virgin Is', 102),
 ('Brunei', 5918),
 ('Bulgaria', 110763),
 ('Burkina Faso', 274387),
 ('Burma', 667905),
 ('Burundi', 26954),
 ('Cabo Verde', 4077),


In [37]:
cluster.shutdown()

In [38]:
#| hide
import nbdev; nbdev.nbdev_export()